In [1]:
from treble_tsdk.tsdk import TSDK, TSDKCredentials
from treble_tsdk import display_data as dd
from treble_tsdk import treble
import random
import json
from tqdm import tqdm
import glob
import math
from typing import List, Tuple
from collections import defaultdict

In [2]:
data_dir = "data"

rooms_data = []
for filename in tqdm(glob.glob(f"{data_dir}/**/*.json", recursive=True), desc="Loading room data"):
    with open(filename, "r") as f:
        data = json.load(f)
        rooms_data.append(data)

tsdk = TSDK(TSDKCredentials.from_file("./creds/tsdk.cred"))


Loading room data: 100%|██████████| 1/1 [00:00<00:00, 6636.56it/s]


== SDK package is up to date ==


In [3]:
project = tsdk.get_or_create_project("ertsi_0")

In [57]:
def sort_edge_loop(room_verts: List[List[float]]) -> List[List[float]]:
    verts = [tuple(v) for v in room_verts]
    adj = defaultdict(list)

    def is_neighbor(a, b, tol=1e-5):
        dx = abs(a[0] - b[0])
        dy = abs(a[1] - b[1])
        return (dx == 0 or dy == 0) and (dx + dy) > tol and (dx + dy) < 10  # Adjust threshold if needed

    # Build adjacency list
    for i, vi in enumerate(verts):
        for j, vj in enumerate(verts):
            if i != j and is_neighbor(vi, vj):
                adj[vi].append(vj)

    # Define direction vectors
    DIRS = {
        (1, 0): 0,   # Right
        (0, 1): 1,   # Up
        (-1, 0): 2,  # Left
        (0, -1): 3   # Down
    }

    def vec_dir(from_pt, to_pt):
        dx = int(math.copysign(1, to_pt[0] - from_pt[0]) if to_pt[0] != from_pt[0] else 0)
        dy = int(math.copysign(1, to_pt[1] - from_pt[1]) if to_pt[1] != from_pt[1] else 0)
        return (dx, dy)

    # Start from lowest-left point
    start = min(verts, key=lambda v: (v[1], v[0]))
    current = start
    prev = (start[0] - 1, start[1])  # Fake previous point to imply initial direction is right
    loop = [current]
    visited = set([current])

    while True:
        # Determine direction
        direction = vec_dir(prev, current)
        # Sort neighbors in clockwise order starting from current direction
        neighbors = adj[current]

        def angle_from_dir(n):
            v = vec_dir(current, n)
            return (DIRS[v] - DIRS[direction]) % 4  # 0 = straight, 1 = right turn, etc.

        neighbors_sorted = sorted(neighbors, key=angle_from_dir)

        for n in neighbors_sorted:
            if n != prev:
                if n == start and len(loop) > 2:
                    return [list(v) for v in loop]  # Closed loop
                if n not in visited or len(loop) >= len(verts):
                    loop.append(n)
                    prev, current = current, n
                    visited.add(n)
                    break
        else:
            break

    return [list(v) for v in loop]

In [ ]:
room_definitions = []
for i, room_data in enumerate(rooms_data):
    edge_points_hexagon = sort_edge_loop(room_data["room_verts"])
    room_height = 3
    room = treble.GeometryDefinitionGenerator.create_polygon_room(
        points_xy=edge_points_hexagon, height_z=room_height, join_wall_layers=True
    )

    room_definitions.append(room)
room_definitions

Room 0: [[-3.0, -2.0], [2.0, -2.0], [2.0, 2.0], [2.0, 5.0], [2.0, 7.0], [2.0, 11.498441696166992], [-3.0, 11.498441696166992], [-3.0, 2.0]]


[GeometryDefinition(geometry_component_count=0.]

In [29]:
generated_rooms = []

print("=== Populating rooms with geometry components ===")
for i, (room_def, room_data) in tqdm(enumerate(zip(room_definitions, rooms_data))):
    room_def.clear_geometry_components()
    placements = [
        treble.GeometryComponentPlacement(
            components=tsdk.geometry_component_library.query(group="desk"),
            preferred_count=room_data["desk_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
        ),
        treble.GeometryComponentPlacement(
            components=tsdk.geometry_component_library.query(group="chair"),
            preferred_count=room_data["chair_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
        )
    ]
    room_def.populate_with_geometry_components(
        components=placements,
        selection_algorithm=treble.ComponentSelectionAlgorithm.random,
    )

    model = project.add_model(f"room_{i}_10", room_def)
    generated_rooms.append(model)

=== Populating rooms with geometry components ===


1it [00:02,  2.30s/it]


In [53]:
random_model = random.choice(generated_rooms)
random_model.plot()

In [80]:
all_materials = tsdk.material_library.get()
database_materials = [
    material for material in all_materials if material["organizationId"] == None
]

all_material_assignments = []
for i, (model, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    layers = {
        "polygon_room_walls": "gypsum/plaster on solid backing",
        "polygon_room_floor": room_data["floor_material"],
        "polygon_room_ceiling": room_data["ceiling_material"],
        "Furniture/Desk": "wood",
        "Furniture/Chair": "upholstered concert chairs",
        "Furniture/Chair A": "upholstered concert chairs",
        "Furniture/Bar Stool": "upholstered concert chairs",
    }
    material_assignment = []

    for layer in model.layer_names:
        if layer in layers:
            search_string = layers[layer]
            matches = [
                m for m in database_materials if search_string.lower() in m.name.lower()
            ]
            if matches:
                material_assignment.append(
                    treble.MaterialAssignment(layer, random.choice(matches))
                )
    all_material_assignments.append(material_assignment)

dd.display(all_material_assignments[-1])

1it [00:00, 1969.16it/s]


                     Material assignment                     
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer name           ┃ Material name                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ polygon_room_ceiling │ Plywood panelling, 1 cm thick      │
│ polygon_room_walls   │ Gypsum/Plaster on solid backing    │
│ Furniture/Desk       │ Wooden pews (100% occupancy)       │
│ polygon_room_floor   │ Linoleum or vinyl on concrete      │
│ Furniture/Chair A    │ Heavily upholstered concert chairs │
│ Furniture/Chair      │ Heavily upholstered concert chairs │
└──────────────────────┴────────────────────────────────────┘

In [ ]:
all_sources = []
all_source_positions = []
all_receivers = []
all_receiver_positions = []

pg = treble.PointsGenerator()

pos_ruleset = treble.PointRuleset(
    min_dist_from_surface=0.5,
    min_dist_from_other_points=2,
)

for room in tqdm(generated_rooms):
    pos = pg.generate_valid_points(
        model=room,
        max_count=2,	
        ruleset=pos_ruleset,
        z_range=(0.5, 1.5)
    )
    all_source_positions.append(pos[0])
    all_receiver_positions.append(pos[1])

    source = treble.Source.make_omni(
        position=pos[0],
        label=f"source_{room.name}",
    )
    receiver = treble.Receiver.make_mono(
        position=pos[1],
        label=f"receiver_{room.name}",
    )
    all_sources.append(source)
    all_receivers.append(receiver)


100%|██████████| 1/1 [00:00<00:00, 27.99it/s]


In [ ]:
sim_type = treble.SimulationType.hybrid
crossover_frequency = 500

sim_defs = []

for i, (room, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    sim_def = treble.SimulationDefinition(
        name=f"simulation_{i}_1",  # unique name of the simulation
        simulation_type=sim_type,  # the type of simulation
        crossover_frequency=crossover_frequency,  # the frequency at which the simulation switches from wavebased to image source
        model=room,
        energy_decay_threshold=60,  # simulation termination criteria - the simulation stops running after -60 dB of energy decay
        receiver_list=[all_receivers[i]],
        source_list=[all_sources[i]],
        material_assignment=all_material_assignments[i],
    )
    sim_defs.append(sim_def)

sim_defs = project.add_simulations(sim_defs)

1it [00:00, 10591.68it/s]


In [ ]:
all_sims = project.get_simulations()
dd.display(all_sims)

                                                       Simulations                                                       
┏━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━┓
┃ Index ┃ Name           ┃ Created at       ┃ Id                                   ┃ Status    ┃ Freq. (Hz) ┃ Rcv ┃ Src ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━┩
│ 0     │ Simulation_1   │ 2025-05-15 16:52 │ 36276580-5214-43ca-bc96-60cce4c07c45 │ Completed │ 25         │ 1   │ 1   │
│ 1     │ simulation_0   │ 2025-05-16 21:46 │ f5440a62-623a-48c8-8cab-fd5be14dce7a │ Created   │ 500        │ 1   │ 1   │
│ 2     │ simulation_0_1 │ 2025-05-16 22:29 │ 8a197128-4ac2-41fb-a5a7-246355f63069 │ Created   │ 500        │ 1   │ 1   │
└───────┴────────────────┴──────────────────┴──────────────────────────────────────┴───────────┴────────────┴─────┴─────┘

In [ ]:
random_sim = random.choice(all_sims)
random_sim.plot()

In [ ]:
runtime_estimate = project.estimate()
dd.display(runtime_estimate)


                               Project bc91d45c-e9f6-410f-ba72-d0fbfffcb844 estimate                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                                                       ┃       Simulation ┃ Estimated runtime ┃   Estimated cost ┃
┃                                            Simulation ┃           status ┃           (hours) ┃         (tokens) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│   Simulation_1 (36276580-5214-43ca-bc96-60cce4c07c45) │        Completed │             0.023 │            0.023 │
│   simulation_0 (f5440a62-623a-48c8-8cab-fd5be14dce7a) │          Created │             0.051 │             0.07 │
│ simulation_0_1 (8a197128-4ac2-41fb-a5a7-246355f63069) │          Created │             0.051 │             0.07 │
├───────────────────────────────────────────────────────┼──────────────────┼───────────────────┼──────────────────┤
│                                                 Total │                  │             0.051 │            0.163 │
└───────────────────────────────────────────────────────┴──────────────────┴───────────────────┴──────────────────┘

In [ ]:
# project.start_simulations()
# project.as_live_progress()

In [ ]:
# for sim in all_sims:
#     sim.download_results(f'results/{sim.name}')

In [ ]:
# results_object = simulation.download_results(f'results/{simulation.name}')

# # begin to explore the results
# results_object.plot()

/home/leo/projects/repos/ertsi/generation/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Download simulation 36276580-5214-43ca-bc96-60cce4c07c45: 100%|██████████| 2/2 tasks


In [ ]:
# isolate the data from a single impulse response
# this_result = results_object.get_mono_ir(source=simulation.sources[0],receiver=simulation.receivers[0])
# this_ir = this_result.data
# this_tvec = this_result.time

# this_result.plot()